In [82]:
from ortools.linear_solver import pywraplp
from datetime import datetime, date
import pandas as pd
import calendar
solver = pywraplp.Solver.CreateSolver('GLOP')

## Variáveis do problema

In [83]:
month_idx = 11
now = datetime.now()
n_days = calendar.monthrange(now.year, month_idx)[1]
weekdays  = [date(now.year, month_idx, d).weekday() for d in range(1, n_days + 1)]
first_weekday = weekdays[0]
n_weeks = len(calendar.monthcalendar(now.year, month_idx))
print(n_weeks)

max_workload = 110

5


In [84]:
residents = [
    { "idx": 0, "name": "André", "type": "R1" },
    { "idx": 1, "name": "Maria", "type": "R1" },
    { "idx": 2, "name": "Cinthya", "type": "R1" },
    { "idx": 3, "name": "Rolando", "type": "R1" },
    { "idx": 4, "name": "Victor", "type": "R1" },
    { "idx": 5, "name": "Lucas", "type": "R2" },
    { "idx": 6, "name": "Giselia", "type": "R2" },
    { "idx": 7, "name": "Pedro", "type": "R2" },
    { "idx": 8, "name": "Bárbara", "type": "R2" },
    { "idx": 9, "name": "Douglas", "type": "R2" },
    { "idx": 10, "name": "Filipe", "type": "R3" },
]

n_res = len(residents)
res_types = list(set([r['type'] for r in residents]))
n_types = len(res_types)

In [85]:
enf_list = ['André', 'Maria', 'Cinthya', 'Rolando', 'Victor']
enf_list = [r for r in residents if r['name'] in enf_list]
enf_list

[{'idx': 0, 'name': 'André', 'type': 'R1'},
 {'idx': 1, 'name': 'Maria', 'type': 'R1'},
 {'idx': 2, 'name': 'Cinthya', 'type': 'R1'},
 {'idx': 3, 'name': 'Rolando', 'type': 'R1'},
 {'idx': 4, 'name': 'Victor', 'type': 'R1'}]

In [86]:
shifts = [
    { "idx": 0, "name": "HRC_dia", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 1, "name": "HRC_noite", "start_time": 19, "end_time": 7, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 2, "name": "ENF", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 3, "name": "CC", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,0,0], "num_res": 2 },
    { "idx": 4, "name": "HRSam", "start_time": 19, "end_time": 24, "frequency": [1,1,1,0,0,0,0], "num_res": 2 },
    { "idx": 5, "name": "Amb", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,0,0], "num_res": 1 },
]

n_shifts = len(shifts)

In [87]:
def convert_date(hour, day=1):
    return datetime.now().replace(day=day, hour=hour-1, minute=0, second=0, microsecond=0)

def get_time(shift):
    start = shift['start_time']
    end = shift['end_time']
    if end < start:
        return convert_date(start), convert_date(end, 2)
    return convert_date(start), convert_date(end)

def check_overlap(s1, s2):
    start1, end1 = get_time(s1)
    start2, end2 = get_time(s2)
    if end1 > start2 and end2 > start1:
        return 1

    return 0

overlaps = [[check_overlap(s1, s2) for s1 in shifts] for s2 in shifts]


In [88]:
for i, shift in enumerate(shifts):
    start, end = get_time(shift)
    shifts[i]['workload'] = (end - start).seconds//3600

## Variáveis de decisão

In [89]:
# Variável binária que guarda a escala do mês
# r: residente
# s: plantão
# d: dia do mês
Xrsd = [[[solver.BoolVar(f'X{r}{s}{d}') for d in range(n_days)] for s in range(n_shifts)] for r in range(n_res)]

# Variável que guarda a carga horária semanal de cada residente
# r: residente
# w: semana
Hrw = [[solver.NumVar(0, max_workload, f'H{r}{w}') for w in range(n_weeks)] for r in range(n_res)]

# Somatório da carga horária semanal de todos os residentes por semana para cada tipo de residente
# t: tipo de residente
# w: semana
Stw = [[solver.NumVar(0, solver.infinity(), f'S{t}{w}') for w in range(n_weeks)] for t in range(n_types)]

## Restrições

0. Ligando variáveis de decisão

In [90]:
# Carga horária semanal por residente
for r, resident in enumerate(residents):
    week_counter = 0
    for d, weekday in enumerate(weekdays):
        if weekday == first_weekday:
            ct = solver.Constraint(0, 0, f"workload - resident {resident['name']} - week {week_counter}")
            ct.SetCoefficient(Hrw[r][week_counter], -1)
            week_counter += 1

        for s, shift in enumerate(shifts):
            ct.SetCoefficient(Xrsd[r][s][d], shift['workload'])

In [91]:
# Somatório da carga horária semanal dos residentes por semana
for w in range(n_weeks):
    for t in range(n_types):
        res_list = [r for r in residents if r['type'] == res_types[t]]
        ct = solver.Constraint(0,0, f"Workload sum - week {w} - res_type {t}")
        ct.SetCoefficient(Stw[t][w], -1)
        for res in res_list:
            ct.SetCoefficient(Hrw[res['idx']][w], 1)

1. Métrica de Justiça

In [92]:
precision = 0.1

for w in range(n_weeks):
    for t in range(n_types):
        res_list = [r for r in residents if r['type'] == res_types[t]]
        N = len(res_list)
        # Limites superiores
        # N*xi - (1+precision) Sum_x <=0 ... para cada i em N
        for res in res_list:
            ct = solver.Constraint(-solver.infinity(), 0, f"Upper limit - week {w} - res_type {t} - resident {res['name']}")
            ct.SetCoefficient(Hrw[res['idx']][w], N)
            ct.SetCoefficient(Stw[t][w], -(1+precision))

        # Limites inferiores
        # N*xi - (1-precision) Sum_x >= 0 ... para cada i em N
        for i in range(0,N):
            ct = solver.Constraint(0, solver.infinity(), f"Lower limit - week {w} - res_type {t} - resident {res['name']}")
            ct.SetCoefficient(Hrw[res['idx']][w], N)
            ct.SetCoefficient(Stw[t][w], -(1-precision))

1. Garantir que tenha o número de residentes necessário em cada plantão

In [93]:
for s, shift in enumerate(shifts):
    for d, weekday in enumerate(weekdays):

        # Find the number of residents depending on weekday
        num_res = 0
        if shift['frequency'][weekday]:
            num_res = shift['num_res']

        ct = solver.Constraint(num_res, num_res, f"Day {d + 1} - {num_res} residents in {shift['name']}")
        soma = ""
        for r in range(n_res):
            ct.SetCoefficient(Xrsd[r][s][d], 1)

2. Garantir que não haja choque de horário na escala

In [94]:
for d in range(n_days):
    for r, resident in enumerate(residents):
        for shift_idx, shift in enumerate(shifts):
            ct = solver.Constraint(0, 1, f"No time overlap in day {d + 1} for resident {resident['name']} - Shift {shift['name']}")
            for s, overlap in enumerate(overlaps[shift_idx]):
                ct.SetCoefficient(Xrsd[r][s][d], overlap)

3. Ambulatório só tem R1

In [95]:
s = 5
shift = shifts[s]
r_mais = [res for res in residents if res['type'] in ["R2", "R3"]]

for d in range(n_days):
    for resident in r_mais:
        r = resident['idx']
        ct = solver.Constraint(0, 0, f"No r+ in ambulatory - day {d + 1} - resident {resident['name']}")
        ct.SetCoefficient(Xrsd[r][s][d], 1)

4. HRC_dia/noite, HRSam, ENF e CC precisa de um R1 e um R2/R3 por turno

In [96]:
for d in range(n_days):
    for s, shift in enumerate(shifts[:-1]):
        ct = solver.Constraint(0, 0, f"R1+R2/R3 in day {d + 1} - shift {shift['name']}")
        for r, resident in enumerate(residents):
            is_r1 = 1 if resident['type'] == 'R1' else -1
            ct.SetCoefficient(Xrsd[r][s][d], is_r1)

5. R3 faz HRC_dia ou CC 1x por semana cada

In [97]:
r3_shifts = [shifts[i] for i in [0, 3]]
r3_list = [res for res in residents if res['type']  == "R3"]

for resident in r3_list:
    r = resident['idx']
    for shift in r3_shifts:
        s = shift['idx']
        week_counter = 0
        for d, weekday in enumerate(weekdays):
            if weekday == first_weekday:
                week_counter += 1
                ct = solver.Constraint(1, 1, f"R3 {resident['name']} working once in shift {shift['name']} - week {week_counter}")
            ct.SetCoefficient(Xrsd[r][s][d], 1)

6. R3 não faz parte dos demais plantões (HRC_noite, amb, enf, hrsam)

In [98]:
not_r3_shifts = [shifts[i] for i in [1, 2, 4, 5]]

ct = solver.Constraint(0, 0, f"R3 residents doesnt work in other shifts")
for resident in r3_list:
    r = resident['idx']
    for shift in not_r3_shifts:
        s = shift['idx']
        for d in range(n_days):
            ct.SetCoefficient(Xrsd[r][s][d], 1)

7. O R1 fica a semana toda na enfermaria

In [99]:
week_counter = 0
s = 2  # ENF

for d, weekday in enumerate(weekdays):
    if weekday == first_weekday:
        resident = enf_list[week_counter]
        r = resident['idx']
        week_counter += 1
    ct = solver.Constraint(1, 1, f"R1 {resident['name']} working on ENF - week {week_counter + 1}")
    ct.SetCoefficient(Xrsd[r][s][d], 1)

8. Máximo de horas de trabalho semanais por residente

In [100]:
max_workload = 110

for r, resident in enumerate(residents):
    week_counter = 0
    for d, weekday in enumerate(weekdays):

        if weekday == first_weekday:
            ct = solver.Constraint(0, max_workload, f"R1 {resident['name']} working between {0} and {max_workload} hors - week {week_counter + 1}")
            week_counter += 1

        for s, shift in enumerate(shifts):
            ct.SetCoefficient(Xrsd[r][s][d], shift['workload'])

9. Máximo 24 horas seguidas de trabalho

In [101]:
# como implementar?

## Função objetivo

In [102]:
objective = solver.Objective()

for t in range(n_types):
    for w in range(n_weeks):
        objective.SetCoefficient(Stw[t][w], 1)

objective.SetMinimization()

In [103]:
solver.NumConstraints()

2766

In [104]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('OTIMA')

OTIMA


In [109]:
r = 10
resident = residents[r]

# for r, resident in enumerate(residents):
print(resident['name'])

for d in range(n_days):
    for s, shift in enumerate(shifts[:-1]):
        if round(Xrsd[r][s][d].solution_value(), 0):
            print(f"DIA {d + 1} - {shift['name']}")

Filipe
DIA 5 - CC
DIA 6 - HRC_dia
DIA 10 - CC
DIA 14 - HRC_dia
DIA 15 - CC
DIA 21 - HRC_dia
DIA 26 - CC
DIA 28 - HRC_dia
DIA 29 - HRC_dia
DIA 30 - CC


In [106]:
week_counter = 0

for r, resident in enumerate(residents):
    if week_counter:
        print(f"Semana {week_counter}: {soma}")
    print(f"\n\n RESIDENTE {resident['name']}")

    week_counter = 0
    soma = 0

    for d, weekday in enumerate(weekdays):

        if weekday == first_weekday:
            if week_counter:
                print(f"Semana {week_counter}: {soma}")
            soma = 0
            week_counter += 1

        for s, shift in enumerate(shifts):
            soma += round(Xrsd[r][s][d].solution_value(), 0) * shift['workload']



 RESIDENTE André
Semana 1: 84.0
Semana 2: 89.0
Semana 3: 84.0
Semana 4: 82.0
Semana 5: 34.0


 RESIDENTE Maria
Semana 1: 84.0
Semana 2: 84.0
Semana 3: 60.0
Semana 4: 60.0
Semana 5: 12.0


 RESIDENTE Cinthya
Semana 1: 87.0
Semana 2: 72.0
Semana 3: 84.0
Semana 4: 84.0
Semana 5: 24.0


 RESIDENTE Rolando
Semana 1: 60.0
Semana 2: 72.0
Semana 3: 84.0
Semana 4: 84.0
Semana 5: 24.0


 RESIDENTE Victor
Semana 1: 72.0
Semana 2: 70.0
Semana 3: 75.0
Semana 4: 65.0
Semana 5: 24.0


 RESIDENTE Lucas
Semana 1: 60.0
Semana 2: 72.0
Semana 3: 72.0
Semana 4: 72.0
Semana 5: 17.0


 RESIDENTE Giselia
Semana 1: 58.0
Semana 2: 65.0
Semana 3: 65.0
Semana 4: 72.0
Semana 5: 12.0


 RESIDENTE Pedro
Semana 1: 53.0
Semana 2: 65.0
Semana 3: 60.0
Semana 4: 58.0
Semana 5: 24.0


 RESIDENTE Bárbara
Semana 1: 72.0
Semana 2: 48.0
Semana 3: 48.0
Semana 4: 41.0
Semana 5: 12.0


 RESIDENTE Douglas
Semana 1: 60.0
Semana 2: 53.0
Semana 3: 58.0
Semana 4: 60.0
Semana 5: 17.0


 RESIDENTE Filipe
Semana 1: 24.0
Semana 2: 24.0

In [107]:
schedule = []

for d, weekday in enumerate(weekdays):
    day_schedule = []
    for s, shift in enumerate(shifts):
        res_list = []
        for r, resident in enumerate(residents):
            if round(Xrsd[r][s][d].solution_value(), 0):
                res_list.append(resident['name'])
        day_schedule.append(', '.join(res_list))
    schedule.append(day_schedule)

In [108]:
cols = [s['name'] for s in shifts]
idx = [f'{d + 1}/{month_idx}' for d in range(n_days)]


pd.DataFrame(schedule, columns=cols, index=idx)

,HRC_dia,HRC_noite,ENF,CC,HRSam,Amb
1/11,"Maria, Giselia","Maria, Pedro","André, Douglas","Cinthya, Lucas","Cinthya, Giselia",Victor
2/11,"Rolando, Bárbara","Maria, Bárbara","André, Lucas","Maria, Douglas","Cinthya, Giselia",Victor
3/11,"Rolando, Giselia","Maria, Lucas","André, Bárbara","Maria, Lucas","Cinthya, Pedro",Victor
4/11,"Rolando, Douglas","Rolando, Pedro","André, Bárbara","Cinthya, Pedro",,Victor
5/11,"Rolando, Bárbara","Victor, Douglas","André, Lucas","Victor, Filipe",,Maria
6/11,"Cinthya, Filipe","Cinthya, Douglas","André, Giselia",,,
7/11,"Cinthya, Bárbara","Cinthya, Pedro","André, Giselia",,,
8/11,"Cinthya, Giselia","André, Lucas","Maria, Pedro","André, Bárbara","Victor, Douglas",Victor
9/11,"Victor, Pedro","Rolando, Lucas","Maria, Douglas","Cinthya, Giselia","André, Pedro",André
10/11,"Rolando, Lucas","André, Pedro","Maria, Giselia","Cinthya, Filipe","Victor, Giselia",Victor
